In [32]:
import pandas as pd

df = pd.read_csv("../api/data.csv")

In [31]:
from elasticsearch import Elasticsearch

# Create a client instance
# For local Elasticsearch (default settings)
from elasticsearch import Elasticsearch

es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "TyE3xKUcfnJT_3VkbC4k"),  # Default username is "elastic"
    verify_certs=False,  # Only in development! Not recommended for production
)
# For remote Elasticsearch with authentication
# es = Elasticsearch(
#     "https://your-elasticsearch-host:9200",
#     basic_auth=("username", "password")
# )

# Verify the connection
if es.ping():
    print("Connected to Elasticsearch")
else:
    print("Could not connect to Elasticsearch")

Connected to Elasticsearch


/home/charbel/anaconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


eyJ2ZXIiOiI4LjE0LjAiLCJhZHIiOlsiMTcyLjE4LjAuMjo5MjAwIl0sImZnciI6IjgxY2QzZTMyMmI3MjIyY2YzMDFmODQ3ZGQyNmIzZTJkZTE3YWQzYjY4YWRlNGY1MTYzMDNjNGViMmNkNTdkNGQiLCJrZXkiOiJ6OTFDNkpNQnlGYllncXFSS093dzpCQWFieXptSFJsU21wckVvREpUcWJRIn0=


In [45]:
# Elasticsearch mappings for the models

USER_MAPPING = {
    "mappings": {
        "properties": {
            "email": {"type": "keyword"},  # exact match for emails
            "name": {"type": "text"},
            "password": {"type": "keyword"},  # we shouldn't store plain passwords in ES
            "embedding": {
                "type": "dense_vector",
                "dims": 768,  # adjust dimension based on your embedding model
            },
        }
    }
}

RECIPE_MAPPING = {
    "mappings": {
        "properties": {
            "id": {"type": "integer"},
            "title": {
                "type": "text",
                "fields": {
                    "keyword": {
                        "type": "keyword"
                    }  # allows both full-text and exact matching
                },
            },
            "ingredients": {"type": "text", "index": True, "analyzer": "standard"},
            "instructions": {"type": "text"},
            "prep_time": {"type": "integer"},
            "cook_time": {"type": "integer"},
            "cuisine": {"type": "keyword"},
            "course": {"type": "keyword"},
            "diet": {"type": "keyword"},
            "image": {"type": "text", "index": False},  # URLs stored as keywords
            "url": {"type": "text", "index": False},
            "embedding": {
                "type": "dense_vector",
                "dims": 768,  # adjust dimension based on your embedding model
                "index": True,
                "similarity": "cosine",
            },
        }
    }
}

FEEDBACK_MAPPING = {
    "mappings": {
        "properties": {
            "email": {"type": "keyword"},
            "input_description": {"type": "text"},
            "input_image": {"type": "keyword"},
            "recipe_id": {"type": "integer"},
            "rating": {"type": "integer"},
            "comment": {"type": "text"},
        }
    }
}

RECIPE_ADD_MAPPING = {
    "mappings": {
        "properties": {
            "id": {"type": "integer"},
            "title": {"type": "text", "fields": {"keyword": {"type": "keyword"}}},
            "ingredients": {"type": "text"},
            "instructions": {"type": "text"},
            "prep_time": {"type": "integer"},
            "cook_time": {"type": "integer"},
            "cuisine": {"type": "keyword"},
            "course": {"type": "keyword"},
            "diet": {"type": "keyword"},
            "image": {"type": "keyword"},
            "url": {"type": "keyword"},
            "embedding": {"type": "dense_vector", "dims": 1536},
            "accepted": {"type": "boolean"},  # Additional field for RecipeAdd
        }
    }
}

USER_REVIEW_MAPPING = {
    "mappings": {
        "properties": {
            "email": {"type": "keyword"},
            "reviews": {
                "type": "nested",  # Using nested type for the array of reviews
                "properties": {
                    "email": {"type": "keyword"},
                    "input_description": {"type": "text"},
                    "input_image": {"type": "keyword"},
                    "recipe_id": {"type": "integer"},
                    "rating": {"type": "integer"},
                    "comment": {"type": "text"},
                },
            },
        }
    }
}


# 3. Modified create_indices function with index settings
async def create_indices(es_client):
    """Create Elasticsearch indices with their mappings."""
    indices = {
        "users": USER_MAPPING,
        "recipes": RECIPE_MAPPING,
        "feedback": FEEDBACK_MAPPING,
        "recipe_submissions": RECIPE_ADD_MAPPING,
        "user_reviews": USER_REVIEW_MAPPING,
    }

    # Add index settings
    index_settings = {
        "number_of_shards": 3,
        "number_of_replicas": 1,
        "refresh_interval": "1s",
    }

    for index_name, mapping in indices.items():
        if not await es_client.indices.exists(index=index_name):
            # Merge settings with mapping
            if "settings" not in mapping:
                mapping["settings"] = index_settings
            else:
                mapping["settings"].update(index_settings)

            await es_client.indices.create(index=index_name, body=mapping)
            print(f"Created index: {index_name}")

In [51]:
from elasticsearch import AsyncElasticsearch  # Change this import

# Create async client instance
es = AsyncElasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "TyE3xKUcfnJT_3VkbC4k"),
    verify_certs=False,
)

/home/charbel/anaconda3/lib/python3.12/site-packages/elasticsearch/_async/client/__init__.py:402: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [52]:
await create_indices(es)

Created index: users
Created index: recipes
Created index: feedback
Created index: recipe_submissions
Created index: user_reviews
